# Paramterized Ethane Example

In [ ]:
import mbuild as mb 
import mbuild.recipes 
import foyer 
import gmso 

from gmso.external import from_parmed

import unyt as u
from pprint import pprint
import warnings 
warnings.filterwarnings('ignore')

## The Ideal MoSDeF Workflow

**mBuild -> Foyer -> GMSO** 

- **mBuild**: Construct the chemical system 
- **Foyer**: Atomtype and create a parameterized system
- **GMSO**: Store the information related to the parametrized chemical system and write out to corresponding file formats.

However, as of right now, integrating **GMSO** into **Foyer** is still a work in progress. Hence, for demonstration purposes, the current workflow to create a typed `gmso.Topology` is  
**mBuild -> Foyer -> Parmed -> GMSO**

In [ ]:
from mbuild.lib.molecules import Ethane

mb_ethane = Ethane()
oplsaa = foyer.Forcefield(name='oplsaa')

pmd_ethane = oplsaa.apply(mb_ethane)
top = from_parmed(pmd_ethane)
top.box = gmso.Box([10, 10, 10])
top.name = "Ethane"

The `Topology` created will contain all relevant information for the chemical system including:
- Each `Site` and corresponding `AtomType`
- Connections (bonds, angle, dihedral) each containing `connection_members` and `ConnectionType`

In [ ]:
for atom in top.sites:
    print('Atom name, type: {}, {}'.format(atom.name, atom.atom_type.name))

print()
for bond in top.bonds:
    print('Bond name, connection members: {}\n {}'.format(bond.name, bond.connection_members))

We can take a closer look at the objects stored in `top`:
- **`Atom`** 
- **`Bond`** (other **`Connection`** objects inherit from the same base class and are similar as a result)
- **`AtomType`** (other **`Potential`** objects inherit from the same base class and are similar as a result)
- **`PotentialExpression`**

**`Atom`**

In [ ]:
pprint(top.sites[0].dict(by_alias=True))

**`Bond`**

In [ ]:
pprint(top.bonds[0].dict(by_alias=True))

**`AtomType`**

In [ ]:
pprint(top.sites[0].atom_type.dict(by_alias=True))

**`PotentialExpression`**

In [ ]:
potential_object = top.sites[0].atom_type.potential_expression
print('Potential expression:')
display(potential_object.expression)
print('Independent variables: {}'.format(potential_object.independent_variables))
print('Parameters: {}'.format(potential_object.parameters))

## Writers

In [ ]:
from gmso.formats.gro import read_gro, write_gro
from gmso.formats.top import write_top
from gmso.formats.lammpsdata import write_lammpsdata, read_lammpsdata

Once the `gmso.Topology` is fully parametrized, we can write it out to various file formats. We currently support GROMACS (`.top` and `.gro`), LAMMPS (`.lammps`), and other file formats.

**GROMACS**

In [ ]:
# Pending new location
write_gro(top, './simulations/gromacs/out.gro')
write_top(top, './simulations/gromacs/out.top')

In [ ]:
# Run a simple energy minimization step
%cd ./simulations/gromacs/
!gmx grompp -v -f em.mdp -p out.top -c out.gro -o em.tpr 
!gmx mdrun -v -deffnm em -s em.tpr
%cd ../../

**LAMMPS**

In [ ]:
# Pending new location 
write_lammpsdata(top, './simulations/lammps/ethane.lammps')

In [ ]:
# Pending new location 
# Run a simple energy minimization step
%cd ./simulations/lammps/
!lmp_mpi -in in.ethane -log minimize.log
%cd ../../

In [ ]:
#lmp_energy = - * u.kcal / u.mol
#gro_energy = - * u.kJ / u.mol

In [ ]:
#lmp.to(u.kJ/u.mol)